# Virtual the growth plate from single-cell transcriptomic data

In [ ]:
import scanpy as sc, anndata as ad

import openTSNE
import boundary_utils as bu

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

In [ ]:
filename = '/home/jovyan/researcher_home/Documents/Tom/Atlas/data/Atlas/Growthplate.h5ad'
adata = sc.read_h5ad(filename)
adata

In [ ]:
sc.tl.pca(adata)

Where n_obs × n_vars = 17493 × 43847 -> PCA components and pseudotime value are defined for each cell

In [ ]:
pca_data = adata.obsm['X_pca']
pca_data.shape

In [ ]:
pseudotime = adata.obs['pseudotime']
pseudotime.shape

In [ ]:
adata.obs['CellType'] = adata.obs['CellType'].cat.remove_unused_categories()

Parameters of the circle boundary function which confines TSNE output

In [ ]:
radius = 5
y_center = 2.5
n_slices = 50

## Bounded embedding during TSNE dimensionality reduction

Adaptation of the "t-SNE Animations" tutorial from creators of openTSNE:

https://github.com/pavlin-policar/openTSNE/blob/master/examples/05_animation.ipynb

In [ ]:
x = pca_data

### TSNE without confined embedding

In [ ]:
def set_boundary_embedding(emb, embeddings):
    embeddings.append(np.array(emb))

We pass a callback that will take the current embedding, make a copy (this is important because the embedding is changed inplace during optimization) and add it to a list. We can also specify how often the callbacks should be called. In this instance, we'll call it at every iteration.

In [ ]:
embeddings = []

tsne = openTSNE.TSNE(
    perplexity=50, metric="cosine", n_jobs=32, verbose=True,
    # The embedding will be appended to the list we defined above, make sure we copy the
    # embedding, otherwise the same object reference will be stored for every iteration
    callbacks=lambda it, err, emb: set_boundary_embedding(emb, embeddings),
    # This should be done on every iteration
    callbacks_every_iters=1,
)

In [ ]:
%time tsne_embedding = tsne.fit(x)

In [ ]:
adata.obsm['X_tsne'] = embeddings[-1]

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))

a = np.linspace(-60, 60, 1000)
b = np.linspace(-60, 60, 1000)
A, B = np.meshgrid(a,b)

ax.contour(A, B, bu.boundary_circle(A, B, radius = radius, y_center = y_center) ,[0])

sc.pl.tsne(adata, color='CellType', ax=ax)

ax.set_aspect('equal')

### TSNE with circular embedding

In [ ]:
buondary_function = lambda x , y : bu.boundary_circle(x, y, radius = radius, y_center = y_center)

def set_boundary_embedding(emb, embeddings, boundary_function):
    outside_boundary = buondary_function(emb[:,0], emb[:,1]) < 0
    if any(outside_boundary):
        output = list(map(lambda point : bu.closest_point_in_function(point, buondary_function), emb[outside_boundary]))
        emb[outside_boundary] = np.array(output)
    embeddings.append(np.array(emb))

We pass a callback that will take the current embedding, make a copy (this is important because the embedding is changed inplace during optimization) and add it to a list. We can also specify how often the callbacks should be called. In this instance, we'll call it at every iteration.

In [ ]:
embeddings = []

tsne = openTSNE.TSNE(
    perplexity=50, metric="cosine", n_jobs=32, verbose=True,
    # The embedding will be appended to the list we defined above, make sure we copy the
    # embedding, otherwise the same object reference will be stored for every iteration
    callbacks=lambda it, err, emb: set_boundary_embedding(emb, embeddings, buondary_function),
    # This should be done on every iteration
    callbacks_every_iters=1,
)

In [ ]:
%time tsne_embedding = tsne.fit(x)

In [ ]:
adata.obsm['X_tsne'] = embeddings[-1]

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))

a = np.linspace(-60, 60, 1000)
b = np.linspace(-60, 60, 1000)
A, B = np.meshgrid(a,b)

ax.contour(A, B, bu.boundary_circle(A, B, radius = 35, y_center = 2.5) ,[0])

sc.pl.tsne(adata, color='CellType', ax=ax)

ax.set_aspect('equal')

Plotting a 3D Scatter Plot in Seaborn

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = adata.obsm['X_tsne'][:,0]
y = adata.obsm['X_tsne'][:,1]
z = adata.obs['pseudotime']

ax.set_xlabel("tSNE1")
ax.set_ylabel("tSNE2")
ax.set_zlabel("Pseudotime")

ax.scatter(x, y, z)

### TSNE slices with circular embedding

cut is used to bin pandas DataFrame and specifically define the bin edges. There is no guarantee about the distribution of items in each bin. In fact, you can define bins in such a way that no items are included in a bin or nearly all items are in a single bin.

https://pbpython.com/pandas-qcut-cut.html

In [ ]:
adata.obs['cut_pseudotime'] = pd.cut(adata.obs['pseudotime'], bins=n_slices, include_lowest=True)

After binning the adata object, we separate each bin and run the TSNE dimentionsality reduction with circular embedding. After, the output of the embedded TSNEs will be concatenated and merged.

 - Concatenation is when we keep all sub elements of each object, and stack these elements in an ordered way.

 - Merging is combining a set of collections into one resulting collection which contains elements from the objects.

https://anndata.readthedocs.io/en/latest/concatenation.html

In [ ]:
df_tsne = pd.DataFrame(columns = ['idx', 'X_tsne_sliced'])

adatas = []

for count, item in enumerate(adata.obs.groupby('cut_pseudotime').indices.items()):
    print(f'Slice number: {count}')
    group, idx = item
    
    if idx.size == 0:
            df_tsne.append({'slice' : count, 'group': group, 'idx': idx, 'X_tsne_sliced': []}, ignore_index=True)
    else:
        sub_adata = adata[idx].copy()
    
        bu.circular_openTSNE(sub_adata, radius = radius, y_center = y_center)

        df_tsne.append({'slice' : count, 'group': group, 'idx': idx, 'X_tsne_sliced': sub_adata.obsm['X_tsne']}, ignore_index=True)
        
        adatas.append(sub_adata)

In [ ]:
sliced_adata = ad.concat(adatas)

 ## Plotting virtual growth plate

In [ ]:
sliced_adata.obs.CellType

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

width = 20
height = 20
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [width, height]
plt.rcParams.update({'font.size': 25})
plt.rcParams['grid.color'] = 'w'
plt.rcParams['grid.linewidth'] = 2

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.set_proj_type('ortho')

x = sliced_adata.obsm['X_tsne'][:,0]
y = sliced_adata.obsm['X_tsne'][:,1]
norm_pseudo = sliced_adata.obs['pseudotime']/sliced_adata.obs['pseudotime'].max()
z = 1 - np.cbrt(norm_pseudo)
celltype = sliced_adata.obs['NandC']
celltype_factorized, celltype_uniques = pd.factorize( celltype )

col = ['#FB8072', '#f0f0f0', '#f0f0f0', '#f0f0f0']
color = pd.Index(col)[celltype_factorized]

ax.set_xlabel("tSNE1", labelpad=30)
ax.set_ylabel("tSNE2", labelpad=30)
ax.set_zlabel("Pseudotime", labelpad=30)

scatter = ax.scatter(x, y, z, s = 120, c = color)

"""                                                                                                                                                    
Scaling is done from here...                                                                                                                           
"""
x_scale=1
y_scale=1
z_scale=2

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0

def short_proj():
  return np.dot(Axes3D.get_proj(ax), scale)

ax.get_proj=short_proj
"""                                                                                                                                                    
to here                                                                                                                                                
"""

legend_labelled = (scatter.legend_elements()[0],list(celltype.unique()))

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*legend_labelled,
                    loc="upper right", title="Cell Type")
ax.add_artist(legend1)

# ax.legend()

ax.view_init(10, 70)

# Now set color to white (or whatever is "invisible")
# pane_color = [229/255,236/255,246/255]
pane_color = [224/255,233/255,245/255]
ax.xaxis.pane.set_facecolor(pane_color)
ax.yaxis.pane.set_facecolor(pane_color)
ax.zaxis.pane.set_facecolor(pane_color)

plt.savefig(f'/home/jovyan/researcher_home/Documents/Tom/Atlas/output/2/GP_onlyprolierative.svg')

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

width = 20
height = 20
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [width, height]
plt.rcParams.update({'font.size': 25})
plt.rcParams['grid.color'] = 'w'
plt.rcParams['grid.linewidth'] = 2

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.set_proj_type('ortho')

x = sliced_adata.obsm['X_tsne'][:,0]
y = sliced_adata.obsm['X_tsne'][:,1]
norm_pseudo = sliced_adata.obs['pseudotime']/sliced_adata.obs['pseudotime'].max()
z = 1 - np.cbrt(norm_pseudo)
celltype = sliced_adata.obs['CellType']
celltype_factorized, celltype_uniques = pd.factorize( celltype )

col = ['#f0f0f0', '#D4A6CD', '#f0f0f0', '#f0f0f0']
color = pd.Index(col)[celltype_factorized]

ax.set_xlabel("tSNE1", labelpad=30)
ax.set_ylabel("tSNE2", labelpad=30)
ax.set_zlabel("Pseudotime", labelpad=30)

scatter = ax.scatter(x, y, z, s = 120, c = color)

"""                                                                                                                                                    
Scaling is done from here...                                                                                                                           
"""
x_scale=1
y_scale=1
z_scale=2

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0

def short_proj():
  return np.dot(Axes3D.get_proj(ax), scale)

ax.get_proj=short_proj
"""                                                                                                                                                    
to here                                                                                                                                                
"""

legend_labelled = (scatter.legend_elements()[0],list(celltype.unique()))

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*legend_labelled,
                    loc="upper right", title="Cell Type")
ax.add_artist(legend1)

# ax.legend()

ax.view_init(10, 70)

# Now set color to white (or whatever is "invisible")
# pane_color = [229/255,236/255,246/255]
pane_color = [224/255,233/255,245/255]
ax.xaxis.pane.set_facecolor(pane_color)
ax.yaxis.pane.set_facecolor(pane_color)
ax.zaxis.pane.set_facecolor(pane_color)

plt.savefig(f'/home/jovyan/researcher_home/Documents/Tom/Atlas/output/2/GP_onlyrestingzone.svg')

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

width = 20
height = 20
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [width, height]
plt.rcParams.update({'font.size': 25})
plt.rcParams['grid.color'] = 'w'
plt.rcParams['grid.linewidth'] = 2

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.set_proj_type('ortho')

x = sliced_adata.obsm['X_tsne'][:,0]
y = sliced_adata.obsm['X_tsne'][:,1]
norm_pseudo = sliced_adata.obs['pseudotime']/sliced_adata.obs['pseudotime'].max()
z = 1 - np.cbrt(norm_pseudo)
celltype = sliced_adata.obs['CellType']
celltype_factorized, celltype_uniques = pd.factorize( celltype )

col = ['#f0f0f0', '#f0f0f0', '#0B0F5E', '#f0f0f0']
color = pd.Index(col)[celltype_factorized]

ax.set_xlabel("tSNE1", labelpad=30)
ax.set_ylabel("tSNE2", labelpad=30)
ax.set_zlabel("Pseudotime", labelpad=30)

scatter = ax.scatter(x, y, z, s = 120, c = color)

"""                                                                                                                                                    
Scaling is done from here...                                                                                                                           
"""
x_scale=1
y_scale=1
z_scale=2

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0

def short_proj():
  return np.dot(Axes3D.get_proj(ax), scale)

ax.get_proj=short_proj
"""                                                                                                                                                    
to here                                                                                                                                                
"""

legend_labelled = (scatter.legend_elements()[0],list(celltype.unique()))

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*legend_labelled,
                    loc="upper right", title="Cell Type")
ax.add_artist(legend1)

# ax.legend()

ax.view_init(10, 70)

# Now set color to white (or whatever is "invisible")
# pane_color = [229/255,236/255,246/255]
pane_color = [224/255,233/255,245/255]
ax.xaxis.pane.set_facecolor(pane_color)
ax.yaxis.pane.set_facecolor(pane_color)
ax.zaxis.pane.set_facecolor(pane_color)

plt.savefig(f'/home/jovyan/researcher_home/Documents/Tom/Atlas/output/2/GP_onlyhyp.svg')

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

width = 20
height = 20
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [width, height]
plt.rcParams.update({'font.size': 25})
plt.rcParams['grid.color'] = 'w'
plt.rcParams['grid.linewidth'] = 2

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.set_proj_type('ortho')

x = sliced_adata.obsm['X_tsne'][:,0]
y = sliced_adata.obsm['X_tsne'][:,1]
norm_pseudo = sliced_adata.obs['pseudotime']/sliced_adata.obs['pseudotime'].max()
z = 1 - np.cbrt(norm_pseudo)
celltype = sliced_adata.obs['CellType']
celltype_factorized, celltype_uniques = pd.factorize( celltype )

col = ['#f0f0f0', '#f0f0f0', '#f0f0f0', '#FFD700']
color = pd.Index(col)[celltype_factorized]

ax.set_xlabel("tSNE1", labelpad=30)
ax.set_ylabel("tSNE2", labelpad=30)
ax.set_zlabel("Pseudotime", labelpad=30)

scatter = ax.scatter(x, y, z, s = 120, c = color)

"""                                                                                                                                                    
Scaling is done from here...                                                                                                                           
"""
x_scale=1
y_scale=1
z_scale=2

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0

def short_proj():
  return np.dot(Axes3D.get_proj(ax), scale)

ax.get_proj=short_proj
"""                                                                                                                                                    
to here                                                                                                                                                
"""

legend_labelled = (scatter.legend_elements()[0],list(celltype.unique()))

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*legend_labelled,
                    loc="upper right", title="Cell Type")
ax.add_artist(legend1)

# ax.legend()

ax.view_init(10, 70)

# Now set color to white (or whatever is "invisible")
# pane_color = [229/255,236/255,246/255]
pane_color = [224/255,233/255,245/255]
ax.xaxis.pane.set_facecolor(pane_color)
ax.yaxis.pane.set_facecolor(pane_color)
ax.zaxis.pane.set_facecolor(pane_color)

plt.savefig(f'/home/jovyan/researcher_home/Documents/Tom/Atlas/output/2/GP_onlypre-hyp.svg')

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

width = 20
height = 20
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [width, height]
plt.rcParams.update({'font.size': 25})
plt.rcParams['grid.color'] = 'w'
plt.rcParams['grid.linewidth'] = 2

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.set_proj_type('ortho')

x = sliced_adata.obsm['X_tsne'][:,0]
y = sliced_adata.obsm['X_tsne'][:,1]
norm_pseudo = sliced_adata.obs['pseudotime']/sliced_adata.obs['pseudotime'].max()
z = 1 - np.cbrt(norm_pseudo)
celltype = sliced_adata.obs['CellType']
celltype_factorized, celltype_uniques = pd.factorize( celltype )

col = ['#FB8072', '#D4A6CD', '#0B0F5E', '#FFD700']
color = pd.Index(col)[celltype_factorized]

ax.set_xlabel("tSNE1", labelpad=30)
ax.set_ylabel("tSNE2", labelpad=30)
ax.set_zlabel("Pseudotime", labelpad=30)

scatter = ax.scatter(x, y, z, s = 120, c = color)

"""                                                                                                                                                    
Scaling is done from here...                                                                                                                           
"""
x_scale=1
y_scale=1
z_scale=2

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0

def short_proj():
  return np.dot(Axes3D.get_proj(ax), scale)

ax.get_proj=short_proj
"""                                                                                                                                                    
to here                                                                                                                                                
"""

legend_labelled = (scatter.legend_elements()[0],list(celltype.unique()))

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*legend_labelled,
                    loc="upper right", title="Cell Type")
ax.add_artist(legend1)

# ax.legend()

ax.view_init(10, 70)

# Now set color to white (or whatever is "invisible")
# pane_color = [229/255,236/255,246/255]
pane_color = [224/255,233/255,245/255]
ax.xaxis.pane.set_facecolor(pane_color)
ax.yaxis.pane.set_facecolor(pane_color)
ax.zaxis.pane.set_facecolor(pane_color)

plt.savefig(f'/home/jovyan/researcher_home/Documents/Tom/Atlas/output/2/GP.svg')

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as clr


width = 20
height = 20
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [width, height]
plt.rcParams.update({'font.size': 30})
plt.rcParams['grid.color'] = 'w'
plt.rcParams['grid.linewidth'] = 2

In [ ]:
gene = 'Ihh'

In [ ]:
from pathlib import Path
folder_path = "/home/jovyan/researcher_home/Documents/Tom/Atlas/output/2/" + gene
Path(folder_path).mkdir(parents=True, exist_ok=True)
#os.chmod(folder_path, 775)

In [ ]:
plotname = folder_path + '/virtual-growth_plate_' + gene

In [ ]:
gene_values = sliced_adata[:, gene].X.toarray()

In [ ]:
vmin = np.percentile(gene_values, 10)
vmax = np.percentile(gene_values, 90)
# vmax=10

In [ ]:
color_scale = ["lightgrey", "blue"]
cmap = clr.LinearSegmentedColormap.from_list('custom cmap', color_scale, N=256)

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.set_proj_type('ortho')

x = sliced_adata.obsm['X_tsne'][:,0]
y = sliced_adata.obsm['X_tsne'][:,1]
norm_pseudo = sliced_adata.obs['pseudotime']/sliced_adata.obs['pseudotime'].max()
z = 1 - np.cbrt(norm_pseudo)
celltype = sliced_adata.obs['CellType']
celltype_factorized, celltype_uniques = pd.factorize( celltype )

col = ['#636dfa', '#ef553b', '#00cc96', '#FF0000']
color = pd.Index(col)[celltype_factorized]

scatter = ax.scatter(x, y, z, s = 120, c = gene_values, cmap = cmap, vmin=vmin, vmax=2)
#scatter = ax.scatter(x, y, z, s = 120, c = gene_values, cmap = cmap)
"""                                                                                                                                                    
Scaling is done from here...                                                                                                                           
"""
x_scale=1
y_scale=1
z_scale=2

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0

def short_proj():
  return np.dot(Axes3D.get_proj(ax), scale)

ax.get_proj=short_proj
"""                                                                                                                                                    
to here                                                                                                                                                
"""

ax.set_title(gene)

ax.set(yticklabels='')
ax.set(xticklabels='')
ax.set(zticklabels='')

ax.view_init(10, 70)

# Now set color to white (or whatever is "invisible")
# pane_color = [229/255,236/255,246/255]
pane_color = [224/255,233/255,245/255]
ax.xaxis.pane.set_facecolor(pane_color)
ax.yaxis.pane.set_facecolor(pane_color)
ax.zaxis.pane.set_facecolor(pane_color)

cbar = plt.colorbar(scatter, shrink=0.35)
cbar.ax.tick_params(size=0, labelsize=30)

filename = plotname + '_vector.svg'
plt.savefig(filename)

In [ ]:
gene = 'Col10a1'

In [ ]:
from pathlib import Path
folder_path = "/home/jovyan/researcher_home/Documents/Tom/Atlas/output/2/" + gene
Path(folder_path).mkdir(parents=True, exist_ok=True)
#os.chmod(folder_path, 775)

In [ ]:
plotname = folder_path + '/virtual-growth_plate_' + gene

In [ ]:
gene_values = sliced_adata[:, gene].X.toarray()

In [ ]:
vmin = np.percentile(gene_values, 10)
vmax = np.percentile(gene_values, 90)
# vmax=10

In [ ]:
color_scale = ["lightgrey", "blue"]
cmap = clr.LinearSegmentedColormap.from_list('custom cmap', color_scale, N=256)

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.set_proj_type('ortho')

x = sliced_adata.obsm['X_tsne'][:,0]
y = sliced_adata.obsm['X_tsne'][:,1]
norm_pseudo = sliced_adata.obs['pseudotime']/sliced_adata.obs['pseudotime'].max()
z = 1 - np.cbrt(norm_pseudo)
celltype = sliced_adata.obs['CellType']
celltype_factorized, celltype_uniques = pd.factorize( celltype )

col = ['#636dfa', '#ef553b', '#00cc96', '#FF0000']
color = pd.Index(col)[celltype_factorized]

scatter = ax.scatter(x, y, z, s = 120, c = gene_values, cmap = cmap, vmin=vmin, vmax=2)
#scatter = ax.scatter(x, y, z, s = 120, c = gene_values, cmap = cmap)
"""                                                                                                                                                    
Scaling is done from here...                                                                                                                           
"""
x_scale=1
y_scale=1
z_scale=2

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0

def short_proj():
  return np.dot(Axes3D.get_proj(ax), scale)

ax.get_proj=short_proj
"""                                                                                                                                                    
to here                                                                                                                                                
"""

ax.set_title(gene)

ax.set(yticklabels='')
ax.set(xticklabels='')
ax.set(zticklabels='')

ax.view_init(10, 70)

# Now set color to white (or whatever is "invisible")
# pane_color = [229/255,236/255,246/255]
pane_color = [224/255,233/255,245/255]
ax.xaxis.pane.set_facecolor(pane_color)
ax.yaxis.pane.set_facecolor(pane_color)
ax.zaxis.pane.set_facecolor(pane_color)

cbar = plt.colorbar(scatter, shrink=0.35)
cbar.ax.tick_params(size=0, labelsize=30)

filename = plotname + '_vector.svg'
plt.savefig(filename)

In [ ]:
gene = 'Pthlh'

In [ ]:
from pathlib import Path
folder_path = "/home/jovyan/researcher_home/Documents/Tom/Atlas/output/2/" + gene
Path(folder_path).mkdir(parents=True, exist_ok=True)
#os.chmod(folder_path, 775)

In [ ]:
plotname = folder_path + '/virtual-growth_plate_' + gene

In [ ]:
gene_values = sliced_adata[:, gene].X.toarray()

In [ ]:
vmin = np.percentile(gene_values, 10)
vmax = np.percentile(gene_values, 90)
# vmax=10

In [ ]:
color_scale = ["lightgrey", "blue"]
cmap = clr.LinearSegmentedColormap.from_list('custom cmap', color_scale, N=256)

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.set_proj_type('ortho')

x = sliced_adata.obsm['X_tsne'][:,0]
y = sliced_adata.obsm['X_tsne'][:,1]
norm_pseudo = sliced_adata.obs['pseudotime']/sliced_adata.obs['pseudotime'].max()
z = 1 - np.cbrt(norm_pseudo)
celltype = sliced_adata.obs['CellType']
celltype_factorized, celltype_uniques = pd.factorize( celltype )

col = ['#636dfa', '#ef553b', '#00cc96', '#FF0000']
color = pd.Index(col)[celltype_factorized]

scatter = ax.scatter(x, y, z, s = 120, c = gene_values, cmap = cmap, vmin=vmin, vmax=2)
#scatter = ax.scatter(x, y, z, s = 120, c = gene_values, cmap = cmap)
"""                                                                                                                                                    
Scaling is done from here...                                                                                                                           
"""
x_scale=1
y_scale=1
z_scale=2

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0

def short_proj():
  return np.dot(Axes3D.get_proj(ax), scale)

ax.get_proj=short_proj
"""                                                                                                                                                    
to here                                                                                                                                                
"""

ax.set_title(gene)

ax.set(yticklabels='')
ax.set(xticklabels='')
ax.set(zticklabels='')

ax.view_init(10, 70)

# Now set color to white (or whatever is "invisible")
# pane_color = [229/255,236/255,246/255]
pane_color = [224/255,233/255,245/255]
ax.xaxis.pane.set_facecolor(pane_color)
ax.yaxis.pane.set_facecolor(pane_color)
ax.zaxis.pane.set_facecolor(pane_color)

cbar = plt.colorbar(scatter, shrink=0.35)
cbar.ax.tick_params(size=0, labelsize=30)

filename = plotname + '_vector.svg'
plt.savefig(filename)

In [ ]:
gene = 'Ucma'

In [ ]:
from pathlib import Path
folder_path = "/home/jovyan/researcher_home/Documents/Tom/Atlas/output/2/" + gene
Path(folder_path).mkdir(parents=True, exist_ok=True)
#os.chmod(folder_path, 775)

In [ ]:
plotname = folder_path + '/virtual-growth_plate_' + gene

In [ ]:
gene_values = sliced_adata[:, gene].X.toarray()

In [ ]:
vmin = np.percentile(gene_values, 10)
vmax = np.percentile(gene_values, 90)
# vmax=10

In [ ]:
color_scale = ["lightgrey", "blue"]
cmap = clr.LinearSegmentedColormap.from_list('custom cmap', color_scale, N=256)

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.set_proj_type('ortho')

x = sliced_adata.obsm['X_tsne'][:,0]
y = sliced_adata.obsm['X_tsne'][:,1]
norm_pseudo = sliced_adata.obs['pseudotime']/sliced_adata.obs['pseudotime'].max()
z = 1 - np.cbrt(norm_pseudo)
celltype = sliced_adata.obs['CellType']
celltype_factorized, celltype_uniques = pd.factorize( celltype )

col = ['#636dfa', '#ef553b', '#00cc96', '#FF0000']
color = pd.Index(col)[celltype_factorized]

scatter = ax.scatter(x, y, z, s = 120, c = gene_values, cmap = cmap, vmin=vmin, vmax=2)
#scatter = ax.scatter(x, y, z, s = 120, c = gene_values, cmap = cmap)
"""                                                                                                                                                    
Scaling is done from here...                                                                                                                           
"""
x_scale=1
y_scale=1
z_scale=2

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0

def short_proj():
  return np.dot(Axes3D.get_proj(ax), scale)

ax.get_proj=short_proj
"""                                                                                                                                                    
to here                                                                                                                                                
"""

ax.set_title(gene)

ax.set(yticklabels='')
ax.set(xticklabels='')
ax.set(zticklabels='')

ax.view_init(10, 70)

# Now set color to white (or whatever is "invisible")
# pane_color = [229/255,236/255,246/255]
pane_color = [224/255,233/255,245/255]
ax.xaxis.pane.set_facecolor(pane_color)
ax.yaxis.pane.set_facecolor(pane_color)
ax.zaxis.pane.set_facecolor(pane_color)

cbar = plt.colorbar(scatter, shrink=0.35)
cbar.ax.tick_params(size=0, labelsize=30)

filename = plotname + '_vector.svg'
plt.savefig(filename)

In [ ]:
gene = 'Pth1r'

In [ ]:
from pathlib import Path
folder_path = "/home/jovyan/researcher_home/Documents/Tom/Atlas/output/2/" + gene
Path(folder_path).mkdir(parents=True, exist_ok=True)
#os.chmod(folder_path, 775)

In [ ]:
plotname = folder_path + '/virtual-growth_plate_' + gene

In [ ]:
gene_values = sliced_adata[:, gene].X.toarray()

In [ ]:
vmin = np.percentile(gene_values, 10)
vmax = np.percentile(gene_values, 90)
# vmax=10

In [ ]:
color_scale = ["lightgrey", "blue"]
cmap = clr.LinearSegmentedColormap.from_list('custom cmap', color_scale, N=256)

fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
ax.set_proj_type('ortho')

x = sliced_adata.obsm['X_tsne'][:,0]
y = sliced_adata.obsm['X_tsne'][:,1]
norm_pseudo = sliced_adata.obs['pseudotime']/sliced_adata.obs['pseudotime'].max()
z = 1 - np.cbrt(norm_pseudo)
celltype = sliced_adata.obs['CellType']
celltype_factorized, celltype_uniques = pd.factorize( celltype )

col = ['#636dfa', '#ef553b', '#00cc96', '#FF0000']
color = pd.Index(col)[celltype_factorized]

scatter = ax.scatter(x, y, z, s = 120, c = gene_values, cmap = cmap, vmin=vmin, vmax=2)
#scatter = ax.scatter(x, y, z, s = 120, c = gene_values, cmap = cmap)
"""                                                                                                                                                    
Scaling is done from here...                                                                                                                           
"""
x_scale=1
y_scale=1
z_scale=2

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0

def short_proj():
  return np.dot(Axes3D.get_proj(ax), scale)

ax.get_proj=short_proj
"""                                                                                                                                                    
to here                                                                                                                                                
"""

ax.set_title(gene)

ax.set(yticklabels='')
ax.set(xticklabels='')
ax.set(zticklabels='')

ax.view_init(10, 70)

# Now set color to white (or whatever is "invisible")
# pane_color = [229/255,236/255,246/255]
pane_color = [224/255,233/255,245/255]
ax.xaxis.pane.set_facecolor(pane_color)
ax.yaxis.pane.set_facecolor(pane_color)
ax.zaxis.pane.set_facecolor(pane_color)

cbar = plt.colorbar(scatter, shrink=0.35)
cbar.ax.tick_params(size=0, labelsize=30)

filename = plotname + '_vector.svg'
plt.savefig(filename)